In [ ]:
from bs4 import BeautifulSoup as bs
from splinter import Browser
import time
import pymongo

In [ ]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=True)

In [ ]:
scrapable = ['Processors', 'Chipsets']
scrape = scrapable[0]

In [ ]:
url = 'https://ark.intel.com/content/www/us/en/ark.html#@'+scrape
csvfilename = f'IntelArk{scrape}.csv'
baseurl = 'https://ark.intel.com/'

browser.visit(url)

rootHTML = bs(browser.html, 'html.parser')

In [ ]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [ ]:
db = client.IntelScrape
collection = db.items

In [ ]:
for div in rootHTML.find_all('div', scrape):
    for link in rootHTML.\
        find_all('div', {'data-parent-panel-key':div['data-panel-key']})[0].\
        find_all('a'):
        browser.visit(baseurl+link['href'])
        time.sleep(1)
        for td in bs(browser.html, 'html.parser').find_all('td', class_='ark-product-name'):
            browser.visit(baseurl+td.a['href'])
            time.sleep(1)
            data = {'scrape':scrape}
            try:
                for c in bs(browser.html, 'html.parser').\
                    find_all('div', class_='arkProductSpecifications')[0].\
                    find_all('span', class_=['label','value']):
                    if c['class'][0] == 'label':
                        temp = c.text.strip().replace('.','_')
                    else:
                        data[temp] = c.text.strip()
            except:
                print('failed '+link['href'])
                continue
            collection.insert_one(data)
            print(td.a.text, end='; ')

In [ ]:
browser.quit()

In [ ]:
import csv
with open(csvfilename,'w',newline='') as csvfile:
    cols = []
    data = db.items.find({'scrape':{'$eq':scrape}})
    for i in data:
        cols.extend([x for x in i.keys() if x not in cols])
    cols.remove('_id')
    cols.remove('scrape')
    
    writer = csv.DictWriter(csvfile, cols)
    writer.writeheader()
    for i in data.rewind():
        i.pop('_id')
        i.pop('scrape')
        writer.writerow(i)